In [ ]:
import cv2
import torch
import numpy as np
from pathlib import Path
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import deque
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
from transformers import pipeline
import datetime
from typing import Dict
import os
import json
from langchain.llms import Ollama
from tensorflow.keras.models import load_model


## OBJECT DETECTION (PERSON)
#### YOLOV5

In [ ]:
import warnings
import torch 
import sys
import os

# Add the yolov5 directory to path
sys.path.append(r"C:\Users\ASUS\Women Safety Ecosystem\yolov5\models\yolov5m.yaml")  

warnings.simplefilter(action='ignore', category=FutureWarning)

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.classes = [0]  # Only detect persons (class 0 in COCO dataset)
model.conf = 0.5  # Confidence threshold
model.cpu()  # Ensure model runs on CPU

# Gender Classification

In [ ]:
# Load gender classification model
gender_model = load_model('cctv_gender_classifier.h5')
gender_classes = ['man', 'woman']

# Initialize DeepSort
tracker = DeepSort(max_age=10, 
                   n_init=3,
                   nms_max_overlap=1.0,
                   max_cosine_distance=0.3,
                   nn_budget=None,
                   override_track_class=None,
                   embedder="mobilenet",
                   half=True,
                   bgr=True,
                   embedder_gpu=False,
                   embedder_model_name=None,
                   embedder_wts=None,
                   polygon=False,
                   today=None)

In [ ]:
# Load the action recognition pipeline
action_pipe = pipeline("image-classification", model="rvv-karma/Human-Action-Recognition-VIT-Base-patch16-224", framework="tf")

def preprocess_body(body_crop, target_size=(126, 126)):
    try:
        body_crop = cv2.resize(body_crop, target_size)
        body_crop = body_crop.astype("float32") / 255.0
        body_crop = img_to_array(body_crop)
        body_crop = np.expand_dims(body_crop, axis=0)
        return body_crop
    except Exception as e:
        print(f"Error in preprocess_body: {e}")
        return None

In [ ]:
# Initialize variables
total_persons = 0
gender_counts = {'man': 0, 'woman': 0}
frame_skip = 2
processing_times = []
fighting_count = 0
lone_woman_flag = False
surrounded_woman_flag = False

# New variables for report generation
sos_events = []
alerts = []
warnings = []
gender_counts_over_time = []

# CCTV Surveillance

In [ ]:
video = cv2.VideoCapture(r"C:\Users\ASUS\Desktop\Important files\Women safety ew\Zinnovation\SIH Videos\WhatsApp Video 2024-12-04 at 11.27.37_9177274c.mp4")
frame_count = 0

# Variables for tracking and detection
frame_skip = 2  # Adjust based on performance needs
total_persons = 0
alerts = []
warnings = []
sos_events = []
gender_counts_over_time = []

# Variables for SOS recording
sos_recording = False
output_video = None
output_filename = ""

# Create a folder to store SOS recordings 
sos_folder = "SOS_Recordings"
if not os.path.exists(sos_folder):
    os.makedirs(sos_folder)

# Detection and tracking parameters
CONFIDENCE_THRESHOLD = 0.25
NMS_THRESHOLD = 0.3

# Initialize DeepSort with default parameters
tracker = DeepSort()  # Using default configuration

def is_night(hour):
    return hour < 6 or hour >= 18

def print_alert(message):
    current_time = datetime.datetime.now()
    alert_info = {
        "message": message,
        "date": current_time.strftime('%Y-%m-%d'),
        "time": current_time.strftime('%I:%M:%S %p'),
        "location": "XYZ"
    }
    alerts.append(alert_info)
    print(f"Alert: {message}")
    print(f"Date: {alert_info['date']}")
    print(f"Time: {alert_info['time']}")
    print(f"Location: {alert_info['location']}")
    print("-------------------")

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue
    
    # Resize frame for faster processing
    frame = cv2.resize(frame, (640, 480))
    
    # Get current date and time
    current_time = datetime.datetime.now()
    time_str = current_time.strftime("%Y-%m-%d %I:%M:%S %p")
    cv2.putText(frame, time_str, (320, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    cv2.putText(frame, time_str, (320, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Determine if it's day or night
    is_night_time = is_night(current_time.hour)
    day_night_str = "Night" if is_night_time else "Day"
    cv2.putText(frame, day_night_str, (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    cv2.putText(frame, day_night_str, (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # YOLOv5 detection
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()

    # Apply Non-Maximum Suppression
    boxes = []
    scores = []
    filtered_detections = []
    
    for det in detections:
        if det[4] >= CONFIDENCE_THRESHOLD:
            boxes.append([det[0], det[1], det[2] - det[0], det[3] - det[1]])
            scores.append(det[4])
            filtered_detections.append(det)
    
    if boxes:
        indices = cv2.dnn.NMSBoxes(
            boxes,
            scores,
            CONFIDENCE_THRESHOLD,
            NMS_THRESHOLD
        )
        
        # Prepare filtered detections for DeepSort
        deepsort_detections = []
        for idx in indices:
            det = filtered_detections[idx]
            x1, y1, x2, y2, conf, cls = det
            deepsort_detections.append(([x1, y1, x2 - x1, y2 - y1], conf, int(cls)))
    
        # Update tracks with filtered detections
        tracks = tracker.update_tracks(deepsort_detections, frame=frame)
    else:
        # Update without detections if none were found
        tracks = tracker.update_tracks([], frame=frame)
    
    # Reset gender counts for each frame
    gender_counts = {'man': 0, 'woman': 0}
    
    # Process tracks
    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        
        # Initialize gender history if not present
        if not hasattr(track, 'gender_history'):
            track.gender_history = []
        
        # Perform gender classification with temporal smoothing
        if len(track.gender_history) < 5:
            body_crop = frame[y1:y2, x1:x2]
            preprocessed_body = preprocess_body(body_crop)
            if preprocessed_body is not None:
                try:
                    gender_conf = gender_model.predict(preprocessed_body)[0]
                    gender_idx = np.argmax(gender_conf)
                    if gender_conf[gender_idx] > 0.7:
                        track.gender_history.append(gender_classes[gender_idx])
                except Exception as e:
                    print(f"Error in gender classification: {e}")
        
        # Use majority voting for stable gender classification
        if track.gender_history:
            track.gender = max(set(track.gender_history), key=track.gender_history.count)
        
        # Update gender counts and draw bounding box
        if hasattr(track, 'gender'):
            gender_counts[track.gender] += 1
        
        # Draw box with different colors based on tracking status
        color = (0, 255, 0) if track.time_since_update == 0 else (0, 165, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = f"ID {track_id}"
        if hasattr(track, 'gender'):
            label += f" ({track.gender})"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Perform action recognition
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_frame)
    action_result = action_pipe(pil_image)
    
    # Check for fighting action
    sos_detected = False
    for item in action_result:
        if item['label'] == 'Fighting' and item['score'] > 0.92:
            sos_detected = True
            sos_event = {
                "type": "Fighting",
                "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
                "location": "XYZ",
                "men_count": gender_counts['man'],
                "women_count": gender_counts['woman']
            }
            sos_events.append(sos_event)
            print_alert("SOS! Fighting detected. Action must be taken.")
            cv2.putText(frame, "ALERT! SOS!!", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            if not sos_recording:
                sos_recording = True
                output_filename = os.path.join(sos_folder, f"{current_time.strftime('%d-%m-%Y _ %H:%M')}@XYZ.mp4")
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                output_video = cv2.VideoWriter(output_filename, fourcc, 30.0, (640, 480))
            break

    # Update and display warnings
    lone_woman_flag = gender_counts['woman'] == 1 and gender_counts['man'] == 0
    surrounded_woman_flag = gender_counts['woman'] == 1 and gender_counts['man'] >= 2

    if not sos_detected:
        if lone_woman_flag:
            warning_msg = "Warning: Lone woman at night" if is_night_time else "Notice: Lone woman at daytime"
            cv2.putText(frame, warning_msg, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            warnings.append({
                "message": warning_msg,
                "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
            })
            print_alert(f"{warning_msg}. There is potential threat.")
        elif surrounded_woman_flag:
            warning_msg = "Warning: Woman surrounded by men" 
            cv2.putText(frame, warning_msg, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 165, 255), 2)
            warnings.append({
                "message": warning_msg,
                "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
                "location": "XYZ"
            })
            print_alert(f"{warning_msg}. There is potential threat.")

    # Record frame if SOS is active
    if sos_recording and output_video is not None:
        output_video.write(frame)

    # Display stats and store counts
    cv2.putText(frame, f"Men: {gender_counts['man']}, Women: {gender_counts['woman']}", 
                (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    gender_counts_over_time.append({
        "timestamp": current_time.strftime("%Y-%m-%d %I:%M:%S %p"),
        "men_count": gender_counts['man'],
        "women_count": gender_counts['woman']
    })

    # Display the frame
    cv2.imshow('Full Body Gender Detection and Action Recognition', frame)
    0
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Release resources
video.release()
if output_video is not None:
    output_video.release()
cv2.destroyAllWindows()
if sos_recording:
    print(f"SOS video saved as: {output_filename}")

In [ ]:
video_frames = [
    {
        "frame_number": 1,
        "detections": [
            {"tracking_id": 1, "gender": "male"},
            {"tracking_id": 2, "gender": "female"}
        ],
    },
    {
        "frame_number": 2,
        "detections": [
            {"tracking_id": 1, "gender": "male"},  # Same male as frame 1
            {"tracking_id": 3, "gender": "female"}  # New female
        ],
    },
    {
        "frame_number": 3,
        "detections": [
            {"tracking_id": 2, "gender": "female"},  # Same female as frame 1
            {"tracking_id": 3, "gender": "female"},  # Same female as frame 2
        ],
    },
]

# Initialize sets to store unique IDs
unique_male_ids = set()
unique_female_ids = set()

# Initialize SOS, alerts, and warnings data
sos_events = [{"timestamp": "2024-12-06 12:00:00 PM"}, {"timestamp": "2024-12-06 12:10:00 PM"}]
alerts = []
warnings = [{"timestamp": "2024-12-06 12:20:00 PM"}, {"timestamp": "2024-12-06 12:30:00 PM"}, {"timestamp": "2024-12-06 12:40:00 PM"}]

# Aggregation function
def aggregate_incidents(incident_list, threshold):
    """
    Aggregates incidents within a threshold time interval.
    """
    if not incident_list:
        return 0

    try:
        incident_times = [datetime.datetime.strptime(event['timestamp'], "%Y-%m-%d %I:%M:%S %p") for event in incident_list]
        incident_times.sort()

        aggregated_count = 1
        for i in range(1, len(incident_times)):
            if (incident_times[i] - incident_times[i - 1]).total_seconds() / 60 > threshold:
                aggregated_count += 1

        return aggregated_count
    except KeyError as e:
        print(f"No valid incidents found with '{e.args[0]}'")
        return 0

# Process each frame to track unique individuals
for frame in video_frames:
    for person in frame['detections']:
        tracking_id = person['tracking_id']
        gender = person['gender']
        
        if gender == "female":
            unique_male_ids.add(tracking_id)
        elif gender == "male":
            unique_female_ids.add(tracking_id)

# Aggregate SOS events, alerts, and warnings
aggregated_sos_events = aggregate_incidents(sos_events, threshold=10)
aggregated_alerts = aggregate_incidents(alerts, threshold=10)
aggregated_warnings = aggregate_incidents(warnings, threshold=10)

# Final counts of unique individuals
total_unique_men = len(unique_male_ids)
total_unique_women = len(unique_female_ids)
total_unique_persons = total_unique_men + total_unique_women

# Generate the updated summary
print("======== Video Analysis Summary ========")
print(f"Aggregated SOS events: {aggregated_sos_events}")
print(f"Aggregated warnings: {aggregated_warnings}")
print(f"Total unique persons: {total_unique_persons}")
print(f"Men: {total_unique_men}") 
print(f"Women: {total_unique_women}") 
print("Location: XYZ")
print(f"Gender counts recorded over time: {len(video_frames)} frames")

# REPORT GENERATION

In [ ]:
import json
import datetime
import requests
from typing import Dict

def generate_security_report(
    total_unique_persons: int,
    total_unique_men: int,
    total_unique_women: int,
    aggregated_sos_events: int,
    aggregated_warnings: int,
    frame_count: int,
    location: str = "XYZ"
) -> str:
    """
    Generate a security report using the Ollama Gemma model based on surveillance data.
    
    Args:
        total_unique_persons: Total number of unique individuals detected
        total_unique_men: Number of unique men detected
        total_unique_women: Number of unique women detected
        aggregated_sos_events: Number of aggregated SOS events
        aggregated_warnings: Number of aggregated warnings
        frame_count: Number of frames analyzed
        location: Location of surveillance
    
    Returns:
        str: Generated security report
    """
    
    # Structure the data for the prompt
    summary_data = {
        "surveillance_summary": {
            "total_frames_analyzed": frame_count,
            "unique_persons": {
                "total": total_unique_persons,
                "men": total_unique_men,
                "women": total_unique_women
            },
            "incidents": {
                "sos_events": aggregated_sos_events,
                "warnings": aggregated_warnings
            },
            "location": location
        }
    }
    
    # Create the prompt for the LLM
    prompt = f"""As a security analysis system, generate a concise but detailed report based on the following surveillance data:
{json.dumps(summary_data, indent=2)}
Generate a professional report covering:
1. Key statistics (total persons detected, gender distribution)
2. Security incidents (SOS events and warnings)
3. Notable patterns or concerns
4. Recommendations based on the findings
Format the report in clear, concise paragraphs suitable for security personnel."""

    try:
        # Prepare the request to Ollama API
        url = 'http://localhost:11434/api/chat'
        payload = {
            "model": "gemma2:2b",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False
        }
        
        # Send request to local Ollama server
        response = requests.post(url, json=payload)
        
        # Check if request was successful
        if response.status_code == 200:
            # Extract the report text from the response
            report = response.json()['message']['content']
            return report
        else:
            raise ConnectionError(f"API request failed with status code {response.status_code}")
    
    except ConnectionError as e:
        print("\nConnection Error: Please ensure Ollama is running locally with these steps:")
        print("1. Install Ollama from: https://ollama.ai/")
        print("2. Run 'ollama serve' in terminal")
        print("3. In another terminal, run 'ollama pull gemma:2b'")
        raise e
    except Exception as e:
        print(f"\nError generating report: {str(e)}")
        raise e

def save_report(report: str, prefix: str = "surveillance_report") -> str:
    """Save the generated report to a file with timestamp."""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{prefix}_{timestamp}.txt"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(report)
    
    return filename

# Example usage
if __name__ == "__main__":
    try:
        # Using dynamic summary data from video analysis
        total_unique_men = len(unique_male_ids)  # From your video analysis data
        total_unique_women = len(unique_female_ids)  # From your video analysis data
        total_unique_persons = total_unique_men + total_unique_women  # From your video analysis data
        aggregated_sos_events = aggregated_sos_events  # From your video analysis data
        aggregated_warnings = aggregated_warnings  # From your video analysis data
        frame_count = len(video_frames)  # Number of frames in the video analysis
        location = "XYZ"  # Update with location from your system, if needed
        
        # Generate the report using dynamic data
        report = generate_security_report(
            total_unique_persons=total_unique_persons,
            total_unique_men=total_unique_men,
            total_unique_women=total_unique_women,
            aggregated_sos_events=aggregated_sos_events,
            aggregated_warnings=aggregated_warnings,
            frame_count=frame_count,
            location=location
        )
        
        # Save and display the report
        filename = save_report(report)
        
        print("\nGenerated Report:")
        print("=" * 50)
        print(report)
        print("\nReport saved to:", filename)
        
    except Exception as e:
        print("Failed to generate report:", str(e))
